### Build a Question Answering application over a Graph Database

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
NEO4J_URI=os.getenv("NEO4J_URI")
NEO4J_USERNAME= os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD= os.getenv("NEO4J_PASSWORD")

In [5]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

/var/folders/c3/wt1h7fz910v7vxy5x88m2h_r0000gn/T/ipykernel_74343/1556602031.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)


In [6]:
## Dataset Moview 
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/ashishkumarsahani/vettura-genai/refs/heads/main/Codes/Week_3/Day_1/graph-dataset/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""

In [7]:
moview_query

"\nLOAD CSV WITH HEADERS FROM\n'https://raw.githubusercontent.com/ashishkumarsahani/vettura-genai/refs/heads/main/Codes/Week_3/Day_1/graph-dataset/movies_small.csv' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') | \n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') | \n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') | \n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n\n"

In [8]:
graph.query(moview_query)

[]

In [9]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Trainer {Organization: STRING, Research: STRING, Name: STRING}
Student {Organization: STRING, Name: STRING}
University {Name: STRING}
User {city: STRING, name: STRING, userId: INTEGER, age: STRING}
Post {content: STRING, timestamp: DATE_TIME}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Trainer)-[:Taught]->(:Student)
(:Trainer)-[:Graduated_From]->(:University)
(:Student)-[:Graduated_From]->(:University)
(:User)-[:POSTED]->(:Post)
(:User)-[:LIKES]->(:User)
(:User)-[:FRIEND]->(:User)
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [10]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name=" Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x161e9d150>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x161e9dde0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True, allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x110f9f580>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x161e9d150>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x161e9dde0>, model_name='Gemma2-9b-It', model

In [12]:
response=chain.invoke({"query":"Who was the director of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:DIRECTED]-(p:Person)
RETURN p.name 

Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director of the movie Casino',
 'result': 'Martin Scorsese was the director of the movie Casino. \n'}

In [14]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:ACTED_IN]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods.  \n'}

In [13]:
response=chain.invoke({"query":"How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)
RETURN count(distinct p)  

Full Context:
[{'count(distinct p)': 1239}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1239 artists. \n'}

In [18]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie)
RETURN count(m) 

Full Context:
[{'count(m)': 3}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in', 'result': '3 \n'}

In [14]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [15]:
from langchain_core.prompts import FewShotPromptTemplate,PromptTemplate

example_prompt=PromptTemplate.from_template(
    "User input:{question}\n Cypher query:{query}"
)

prompt=FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question","schema"]
)

In [16]:
print(prompt.format(question="How many artists are there?", schema="foo"))

KeyError: 'name'